In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/alchemy/unlabelled.csv
/kaggle/input/alchemy/11_word_set.csv
/kaggle/input/alchemy/new_11_word_set.csv
/kaggle/input/alchemy/trainingset.csv
/kaggle/input/alchemy/compressed_documents_2.csv.gzip
/kaggle/input/alchemy/all_fragments.csv
/kaggle/input/alchemy/compressed_documents.csv.gzip


In [3]:
# Removes numbers from the text
def remove_numbers(list):
    pattern = '[0-9]'
    list = [re.sub(pattern, '', i) for i in list]
    return list

# Retrieves all fragments from a text sample
def get_fragments(text):
    #Turns the file contents into a list of words, and makes them all lowercase
    casedtext = re.findall(r'\w+', text)
    lowertext = [item.lower() for item in casedtext]
    lowertext = [value for value in lowertext if value != "page" and value != "image"]
    lowertext = remove_numbers(lowertext)

    # Number of words being searched around each term occurrence
    neighbors_range = 5

    # Words that equate to mercury
    mercury_terms = ['argent', 'mercury', 'quicksilver']

    text_fragments = []
    fragment_sample = []

    # Checks if mercury appears in the input
    def check_mercury(word):
      for i in mercury_terms: 
        if i in word: 
          return True
      return False

    # Iterates through each word in the text, and any of the words contains a mercury term, then it creates a neighborhood (the 5 words before + the term + the 5 words after). This tracks how many times salt or sulphur occur in the same set of words as mercury.
    # MERCURY
    for i in range(len(lowertext)):
      if check_mercury(lowertext[i]):
          neighbors = ' '.join(lowertext[i-neighbors_range:i+neighbors_range + 1])
          text_fragments.append(neighbors)

    # SULPHUR
    for i in range(len(lowertext)):
      if 'sulphur' in lowertext[i]: 
        neighbors = ' '.join(lowertext[i-neighbors_range:i+neighbors_range+ 1])
        text_fragments.append(neighbors)

    # SALT
    for i in range(len(lowertext)):
      if 'salt' in lowertext[i]: 
        neighbors = ' '.join(lowertext[i-neighbors_range:i+neighbors_range+ 1])
        text_fragments.append(neighbors)
    
    return text_fragments

In [4]:
# Opens datasets
df = pd.read_csv('/kaggle/input/alchemy/compressed_documents.csv.gzip',compression='gzip')
df

df2 = pd.read_csv('/kaggle/input/alchemy/compressed_documents_2.csv.gzip',compression='gzip')

# Combines the two datasets
df = pd.concat([df, df2])
df

,Unnamed: 0,Title,Date,Publisher,Text
0,0,"Medicinal experiments, or, A collection of cho...",1693-01-01,NaN,Image 1 (unnumbered page) Medicinal Experiment...
1,1,Chymical secrets and rare experiments in physi...,1683-01-01,NaN,Image 1 (unnumbered page) Unnumbered page Chym...
2,2,Observations on the mineral waters of France m...,1684-01-01,NaN,Image 1 (unnumbered page) Unnumbered page OBSE...
3,3,A letter in answer to certain quæries and obje...,1670-01-01,NaN,Image 1 (unnumbered page) Unnumbered page A LE...
4,4,"Paracelsus his Aurora, & treasure of the philo...",1659-01-01,NaN,full_text> Image 1 (unnumbered page) Unnumbe...
...,...,...,...,...,...
99,99,"Vindiciæ literarum, the schools guarded, or, T...",1655-01-01,NaN,Image 1 (unnumbered page) Unnumbered page Vind...
100,100,"Belphegor, or, The marriage of the Devil a tra...",1691-01-01,NaN,Image 1 (unnumbered page) Unnumbered page BELP...
101,101,Tryon's letters upon several occasions ... by ...,1700-01-01,NaN,Image 1 (unnumbered page) Unnumbered page TRYO...
102,102,"The anatomy of human bodies, comprehending the...",1694-01-01,NaN,Image 1 (unnumbered page) Unnumbered page ...


In [5]:
# Creates the fragments column in dataframe
fragments = []

for i in df['Text']:
    fragments.append(get_fragments(i))

df['Fragments'] = fragments

In [6]:
# Removes irrelevant columns
df = df.drop(['Unnamed: 0', 'Publisher','Text'], axis = 1)
df

,Title,Date,Fragments
0,"Medicinal experiments, or, A collection of cho...",1693-01-01,[infuse one dram of clean quicksilverb all nig...
1,Chymical secrets and rare experiments in physi...,1683-01-01,[exuberatum by dr clodius mercury of all meta...
2,Observations on the mineral waters of France m...,1684-01-01,[ whether they caus d mercury sublimate dissol...
3,A letter in answer to certain quæries and obje...,1670-01-01,[a spagyrical physician these are mercury sulp...
4,"Paracelsus his Aurora, & treasure of the philo...",1659-01-01,[many have endeavoured to coagulate mercury an...
...,...,...,...
99,"Vindiciæ literarum, the schools guarded, or, T...",1655-01-01,[lystra he was taken for mercury the god of el...
100,"Belphegor, or, The marriage of the Devil a tra...",1691-01-01,[qualities of salt sulphur and mercury i once ...
101,Tryon's letters upon several occasions ... by ...,1700-01-01,[the ancient philosophers called sulphur mercu...
102,"The anatomy of human bodies, comprehending the...",1694-01-01,[through in small drops like quicksilver strai...


In [7]:
# Saves data
df.to_csv('all_fragments.csv')